In [150]:
import pandas as pd
import numpy as np
import boto3
import os
import datetime


In [151]:
# accessing keys from .env file
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_REGION_NAME = os.get.env("AWS_REGION_NAME")


In [152]:
region_name = 'eu-west-1' 
bucket_name = 'csv-files-parquet'
file_key = 'dummy_boat_data.csv'

# Create an S3 client
s3 = boto3.client('s3', 
                  aws_access_key_id=AWS_ACCESS_KEY_ID, 
                  aws_secret_access_key=AWS_SECRET_ACCESS_KEY, 
                  region_name=AWS_REGION_NAME)

# Read CSV file from S3 into a Pandas DataFrame
try:
    # Use 's3.get_object' to get the object and 'pd.read_csv' to read it into a DataFrame
    obj = s3.get_object(Bucket=bucket_name, Key=file_key)
    df = pd.read_csv(obj['Body'])
    
    # Now 'df' contains your CSV data
    print(df.head())  # Display the first few rows of the DataFrame
except Exception as e:
    print(f"Error: {e}")

    Timestamp speed_over_ground    Longitude   Latitude engine_fuel_rate
0  1675119600              3.08  -119.605263  35.162237            14.92
1  1675119601              4.11  -118.857776  35.228734            21.02
2  1675119602              4.04  -118.110278  35.295227            20.53
3  1675119603               3.5  -117.362786  35.361725            17.21
4  1675119604             ERROR  -116.615295  35.428215            17.38


In [191]:
def import_csv(bucket_name, file_key):
    s3 = boto3.client('s3', 
                    aws_access_key_id= os.getenv("AWS_ACCESS_KEY_ID"), 
                    aws_secret_access_key= os.getenv("AWS_SECRET_ACCESS_KEY"), 
                    region_name= os.getenv("AWS_REGION_NAME"))

    # Read CSV file from S3 into a Pandas DataFrame
    try:
        # Use 's3.get_object' to get the object and 'pd.read_csv' to read it into a DataFrame
        obj = s3.get_object(Bucket=bucket_name, Key=file_key)
        df = pd.read_csv(obj['Body'])
        
    except Exception as e:
        print(f"Error: {e}")

    return df

In [192]:
import_csv('csv-files-parquet', 'dummy_boat_data.csv' )

,Timestamp,speed_over_ground,Longitude,Latitude,engine_fuel_rate
0,1675119600,3.08,-119.605263,35.162237,14.92
1,1675119601,4.11,-118.857776,35.228734,21.02
2,1675119602,4.04,-118.110278,35.295227,20.53
3,1675119603,3.5,-117.362786,35.361725,17.21
4,1675119604,ERROR,-116.615295,35.428215,17.38
...,...,...,...,...,...
995,1675120595,16.18,89.499749,51.884541,81.06
996,1675120596,16.26,89.629043,ERROR,81.49
997,1675120597,NaN,89.758342,53.029681,82.7
998,1675120598,15.93,89.887628,53.602249,79.4


In [154]:
# Testing to write to s3 bucket

# create a sample data frame

books_df = pd.DataFrame(
    data={"Title": ["Book I", "Book II", "Book III"], "Price": [56.6, 59.87,74.54]},
    columns=["Title", "Price"]
)

books_df.head()

,Title,Price
0,Book I,56.60
1,Book II,59.87
2,Book III,74.54


In [155]:
# entering the dataframe into another folder on AWS s3

books_df.to_parquet(
    "s3://new-parquet-files/books.parquet",
    index=False,
    storage_options={
        "key": AWS_ACCESS_KEY_ID,
        "secret":AWS_SECRET_ACCESS_KEY
    }
)

In [201]:
def upload_file(parquet_file, bucket_name):

    s3_resource = boto3.resource(
        "s3",
        region_name = os.getenv("AWS_REGION_NAME"),
        aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY"))

    try:
        s3_resource.Bucket(bucket_name).upload_file(
            Filename = parquet_file,
            Key = os.path.basename(parquet_file))
    
    except Exception as e:
        print(f"Error: {e}")
    
    

In [200]:
cleaned_file = CsvCleaner.clean_file("../src/Data/dummy_boat_data.csv")

Total rows removed: 5


/tmp/ipykernel_1187/54348595.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = pd.to_datetime(df[col_name], unit="s")


In [203]:
upload_file(cleaned_file, "new-parquet-files" )

Error: Failed to upload ../src/Data/dummy_boat_data.parquet to -parquet-files/dummy_boat_data.parquet: An error occurred (NoSuchBucket) when calling the PutObject operation: The specified bucket does not exist


In [156]:
df.head()

,Timestamp,speed_over_ground,Longitude,Latitude,engine_fuel_rate
0,1675119600,3.08,-119.605263,35.162237,14.92
1,1675119601,4.11,-118.857776,35.228734,21.02
2,1675119602,4.04,-118.110278,35.295227,20.53
3,1675119603,3.5,-117.362786,35.361725,17.21
4,1675119604,ERROR,-116.615295,35.428215,17.38


In [157]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Timestamp          990 non-null    object
 1   speed_over_ground  990 non-null    object
 2   Longitude          990 non-null    object
 3   Latitude           990 non-null    object
 4   engine_fuel_rate   990 non-null    object
dtypes: object(5)
memory usage: 39.2+ KB


In [158]:
#class CsvCleaner:
    #@staticmethod
def timestamp_clean(df, col_name):
    # convert the column to numeric with any errors(for example strings or letter) to NaN
    df[col_name] = pd.to_numeric(df[col_name], errors="coerce")

    # remove nan values
    df.dropna(subset=[col_name], inplace=True)

    # Calculate the initial number of rows
    initial_rows = df.shape[0]
    
    #Filter out rows with "Timestamp" values not containing 10 digits
    df = df[df[col_name].apply(lambda x: len(str(int(x))) == 10)]

    #calculate how many rows removed
    rows_removed = initial_rows - df.shape[0]
    
    # Print the number of rows removed
    print(f"{rows_removed} rows were removed.")

    # Convert the Unix timestamp to datetime with seconds
    df[col_name] = pd.to_datetime(df[col_name], unit="s")

    # Sort the DataFrame by the timestamp column
    df = df.sort_values(by=col_name)

    return df

In [159]:
cleaned_df = timestamp_clean(df, "Timestamp")

5 rows were removed.


/tmp/ipykernel_1187/2790770672.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = pd.to_datetime(df[col_name], unit="s")


In [161]:
cleaned_df

,Timestamp,speed_over_ground,Longitude,Latitude,engine_fuel_rate
0,2023-01-30 23:00:00,3.08,-119.605263,35.162237,14.92
1,2023-01-30 23:00:01,4.11,-118.857776,35.228734,21.02
2,2023-01-30 23:00:02,4.04,-118.110278,35.295227,20.53
3,2023-01-30 23:00:03,3.5,-117.362786,35.361725,17.21
4,2023-01-30 23:00:04,ERROR,-116.615295,35.428215,17.38
...,...,...,...,...,...
995,2023-01-30 23:16:35,16.18,89.499749,51.884541,81.06
996,2023-01-30 23:16:36,16.26,89.629043,ERROR,81.49
997,2023-01-30 23:16:37,NaN,89.758342,53.029681,82.7
998,2023-01-30 23:16:38,15.93,89.887628,53.602249,79.4


In [162]:
print(cleaned_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 980 entries, 0 to 999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Timestamp          980 non-null    datetime64[ns]
 1   speed_over_ground  970 non-null    object        
 2   Longitude          970 non-null    object        
 3   Latitude           970 non-null    object        
 4   engine_fuel_rate   970 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 45.9+ KB
None


In [163]:
def clean_columns(df, col_name, low, high):
    
    # Convert column to numeric, making errors to Nan instead
    df[col_name] = pd.to_numeric(df[col_name], errors="coerce")
    
    # Calculate the initial number of rows
    initial_rows = df.shape[0]

    df.loc[~df[col_name].between(low, high), col_name] = float('nan')

    # Calculate the number of rows removed
    rows_removed = initial_rows - df.shape[0]

    # Print the number of rows removed
    print(f"{rows_removed} rows were removed.")


    return df


In [164]:
column = clean_columns(cleaned_df, "Longitude", -180, 180)

0 rows were removed.


In [165]:
column.head()

,Timestamp,speed_over_ground,Longitude,Latitude,engine_fuel_rate
0,2023-01-30 23:00:00,3.08,-119.605263,35.162237,14.92
1,2023-01-30 23:00:01,4.11,-118.857776,35.228734,21.02
2,2023-01-30 23:00:02,4.04,-118.110278,35.295227,20.53
3,2023-01-30 23:00:03,3.5,-117.362786,35.361725,17.21
4,2023-01-30 23:00:04,ERROR,-116.615295,35.428215,17.38


In [166]:
column.info()

<class 'pandas.core.frame.DataFrame'>
Index: 980 entries, 0 to 999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Timestamp          980 non-null    datetime64[ns]
 1   speed_over_ground  970 non-null    object        
 2   Longitude          955 non-null    float64       
 3   Latitude           970 non-null    object        
 4   engine_fuel_rate   970 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 45.9+ KB


In [167]:
column.loc[(column["Longitude"] < -180) | (column["Longitude"] > 180)]

,Timestamp,speed_over_ground,Longitude,Latitude,engine_fuel_rate


In [168]:
print(f"Number of Cells with 'ERROR' in Longitude: {(column['Longitude'] == 'ERROR').sum()}")


Number of Cells with 'ERROR' in Longitude: 0


In [169]:
if column["Longitude"].isnull().any():
    nan_count = df["Longitude"].isnull().sum()
    print(f"There are {nan_count} NaN values in the column.")
else:
    print("There are no NaN values in the column.")

There are 10 NaN values in the column.


In [171]:
speed_over_ground = clean_columns(column, "speed_over_ground", 0,20)

0 rows were removed.


In [172]:
speed_over_ground

,Timestamp,speed_over_ground,Longitude,Latitude,engine_fuel_rate
0,2023-01-30 23:00:00,3.08,-119.605263,35.162237,14.92
1,2023-01-30 23:00:01,4.11,-118.857776,35.228734,21.02
2,2023-01-30 23:00:02,4.04,-118.110278,35.295227,20.53
3,2023-01-30 23:00:03,3.50,-117.362786,35.361725,17.21
4,2023-01-30 23:00:04,NaN,-116.615295,35.428215,17.38
...,...,...,...,...,...
995,2023-01-30 23:16:35,16.18,89.499749,51.884541,81.06
996,2023-01-30 23:16:36,16.26,89.629043,ERROR,81.49
997,2023-01-30 23:16:37,NaN,89.758342,53.029681,82.7
998,2023-01-30 23:16:38,15.93,89.887628,53.602249,79.4


In [173]:
speed_over_ground.loc[(speed_over_ground["speed_over_ground"] < 0) | (column["speed_over_ground"] > 20)]

,Timestamp,speed_over_ground,Longitude,Latitude,engine_fuel_rate


In [174]:
print(f"Number of Cells with 'ERROR' in speed over ground: {(speed_over_ground['speed_over_ground'] == 'ERROR').sum()}")

if speed_over_ground["speed_over_ground"].isnull().any():
    print("There are NaN values in the column.")
else:
    print("There are no NaN values in the column.")

Number of Cells with 'ERROR' in speed over ground: 0
There are NaN values in the column.


In [179]:
class CsvCleaner:
    @staticmethod
    def timestamp_clean(df, col_name):
        # convert the column to numeric with any errors(for example strings or letter) to NaN
        df[col_name] = pd.to_numeric(df[col_name], errors="coerce")
        
        df.dropna(subset=[col_name], inplace=True)

        # Calculate the initial number of rows
        initial_rows = df.shape[0]
        
        #Filter out rows with "Timestamp" values not containing 10 digits
        df = df[df[col_name].apply(lambda x: len(str(int(x))) == 10)]

        #calculate how many rows removed
        rows_removed = initial_rows - df.shape[0]

        # Convert the Unix timestamp to datetime with seconds
        df[col_name] = pd.to_datetime(df[col_name], unit="s")

        # Sort the DataFrame by the timestamp column
        df = df.sort_values(by=col_name)

        return df, rows_removed
    
    @staticmethod
    def clean_columns(df, col_name, low, high):
    
        # Convert column to numeric, making errors to Nan instead
        df[col_name] = pd.to_numeric(df[col_name], errors="coerce")

        # Calculate the initial number of rows
        initial_rows = df.shape[0]

        df.loc[~df[col_name].between(low, high), col_name] = float('nan')

        # Calculate the number of rows removed
        rows_removed = initial_rows - df.shape[0]

        #df[col_name] = df[col_name].interpolate()

        return df, rows_removed
    
    @staticmethod
    def clean_file(csv_file):
        # Read the CSV file into a pandas DataFrame
        df = pd.read_csv(csv_file)
        total_rows_removed = 0

        # Clean the DataFrame
        for col in df.columns:
            if "Timestamp" in col:
                df, rows_removed = CsvCleaner.timestamp_clean(df, col)
                total_rows_removed += rows_removed
            
            if "speed_over_ground" in col:
                low = 0
                high = 100
                df, rows_removed = CsvCleaner.clean_columns(df, col, low, high)
                total_rows_removed += rows_removed
            
            if "Longitude" in col:
                low = -180
                high = 180
                df, rows_removed = CsvCleaner.clean_columns(df, col, low, high)
                total_rows_removed += rows_removed

            if "Latitude" in col:
                low = -90
                high = 90
                df, rows_removed = CsvCleaner.clean_columns(df, col, low, high)
                total_rows_removed += rows_removed

            if "engine_fuel_rate" in col:
                low = 0
                high = 100
                df,rows_removed = CsvCleaner.clean_columns(df, col, low, high)
                total_rows_removed += rows_removed

        # Save the cleaned DataFrame as a Parquet file
        cleaned_parquet_file = csv_file.replace(".csv", ".parquet")
        df.to_parquet(cleaned_parquet_file, index=False)

        print(f"Total rows removed: {total_rows_removed}")

        return cleaned_parquet_file

In [180]:
cleaned_file = CsvCleaner.clean_file("../src/Data/dummy_boat_data.csv")

Total rows removed: 5


/tmp/ipykernel_1187/54348595.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = pd.to_datetime(df[col_name], unit="s")


In [181]:
pd.read_parquet(cleaned_file, engine="pyarrow")

,Timestamp,speed_over_ground,Longitude,Latitude,engine_fuel_rate
0,2023-01-30 23:00:00,3.08,-119.605263,35.162237,14.92
1,2023-01-30 23:00:01,4.11,-118.857776,35.228734,21.02
2,2023-01-30 23:00:02,4.04,-118.110278,35.295227,20.53
3,2023-01-30 23:00:03,3.50,-117.362786,35.361725,17.21
4,2023-01-30 23:00:04,NaN,-116.615295,35.428215,17.38
...,...,...,...,...,...
975,2023-01-30 23:16:35,16.18,89.499749,51.884541,81.06
976,2023-01-30 23:16:36,16.26,89.629043,NaN,81.49
977,2023-01-30 23:16:37,NaN,89.758342,53.029681,82.70
978,2023-01-30 23:16:38,15.93,89.887628,53.602249,79.40
